In [1]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [2]:
twinewsScores = getTwinewsScores()
twinewsRankings = getTwinewsRankings()

twinews scores initialised.


In [3]:
scores = list(twinewsScores.find())
bp(scores)

[
  { _id: 5e7cd4158175111c4825e975, id: lda-aa8d5, metric: ndcg, score: 0.25 },
  { _id: 5e7cd41d8175111c4825e978, id: lda-aa8d5, metric: ndcg@10, score: 0.03 },
  ...,
  { _id: 5ea9860e06cc3c639430ecad, id: combin-01199, metric: jacc-snov@100, score: 0.83 },
  { _id: 5ea989279ec33c5ca39b4dc7, id: combin-1b17f, metric: swjacc-nov@100, score: 0.95 }
]


In [4]:
# We filter scores:
nScores = []
errors = []
eliminated = []
for row in pb(scores):
    try:
        if twinewsRankings.getMeta(row['id'])['maxUsers'] is None:
            nScores.append(row)
        else:
            eliminated.append(row['id'])
    except Exception as e:
        errors.append(row['id'])
print("errors: " + b(errors))
print("eliminated: " + b(eliminated))
print("nScores: " + b(nScores))

  0% [                    ]
 10% [==                  ] (1m 21.72s left)
 20% [====                ] (1m 12.68s left)
 30% [======              ] (1m 3.093s left)
 40% [========            ] (54.3s left)
 50% [==========          ] (45.46s left)
 60% [============        ] (36.453s left)
 70% [==============      ] (27.368s left)
 80% [================    ] (18.232s left)
 90% [==================  ] (9.086s left)
100% [====================] (total duration: 1m 30.769s, mean duration: 0.007s)
errors: [ ]
eliminated: [ tfidf-d0454, tfidf-d0454, ..., tfidf-aed6e, tfidf-aed6e ]
nScores: [
  { _id: 5e7cd4158175111c4825e975, id: lda-aa8d5, metric: ndcg, score: 0.25 },
  { _id: 5e7cd41d8175111c4825e978, id: lda-aa8d5, metric: ndcg@10, score: 0.03 },
  ...,
  { _id: 5ea9860e06cc3c639430ecad, id: combin-01199, metric: jacc-snov@100, score: 0.83 },
  { _id: 5ea989279ec33c5ca39b4dc7, id: combin-1b17f, metric: swjacc-nov@100, score: 0.95 }
]


In [5]:
scores = nScores

In [8]:
metrics = set()
for row in scores:
    metrics.add(row['metric'])
bp(metrics)

{ bm25-ser@100, div@100, ..., topic-snov@100, wtfidf-ser@100 }


In [14]:
allowedMetrics = \
{
    ### Ranking accuracy:
    # 'ndcg', 'ndcg@10', 'ndcg@100', 'map', 'mrr', 'p@10', 'p@100',
    ### Diversity:
    # 'div@100', 'topic-div@100', 'jacc-div@100', 'swjacc-div@100', 'style-div@100',
    ### Novelty:
    # 'nov@100', 'topic-nov@100', 'jacc-nov@100', 'swjacc-nov@100', 'style-nov@100',
    # ### Strict novelty:
    'snov@100', 'topic-snov@100', 'jacc-snov@100', 'swjacc-snov@100', 'style-snov@100',
    ### Serendipity:
    # 'tfidf-ser@100', 'wtfidf-ser@100', 'bm25-ser@100', 'jacc-ser@100', 'style-ser@100',
}

In [15]:
data = []
for metric in metrics:
    if metric in allowedMetrics:
        mini = None
        miniId = None
        maxi = None
        maxiId = None
        for row in scores:
            score = row['score']
            currentMetric = row['metric']
            if currentMetric == metric:
                if mini is None or score < mini: # and ((not "ser@" in metric) or score > 0.0001)
                    mini = score
                    miniId = row['id']
                if maxi is None or score > maxi:
                    maxi = score
                    maxiId = row['id']
        floatTruncation = 2
        nmini = truncateFloat(mini, 1)
        if nmini < 0.0:
            nmini = 0.0
        nmaxi = truncateFloat(maxi, 1) + 0.2
        if nmaxi > 1.0:
            nmaxi = 1.0
        data.append\
        (
            {
                "metric": metric,
                "min": truncateFloat(mini, floatTruncation), "max": truncateFloat(maxi, floatTruncation),
                "nmin": nmini, "nmax": nmaxi,
                "minId": miniId, "maxId": maxiId,
            }
        )
displayDicts(data, start=['metric', 'minId', 'min', 'nmin', 'maxId', 'max', 'nmax'], blue={'min', 'max'}, green=['maxId', 'minId'], red=['nmin', 'nmax'])

,metric,minId,min,nmin,maxId,max,nmax
0,snov@100,tfidf-4b89a,0.87,0.8,worst-f3f99,0.94,1
1,swjacc-snov@100,bm25-c84fd,0.91,0.9,bm25-c4813,0.95,1
2,style-snov@100,dbert-ft-fdc3b,0.25,0.2,worst-f3f99,0.46,0.6
3,jacc-snov@100,combin-75494,0.83,0.8,worst-f3f99,0.85,1
4,topic-snov@100,nmf-17852,0.38,0.3,worst-f3f99,0.68,0.8
